In [59]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


### 1. Load data

In [60]:
import pandas as pd
train_df = pd.read_csv('../data/vn30/ACB_train.csv')
train_df.head(10)
train_df.drop(['time'], axis=1, inplace=True)

### 2. Proces Data

In [61]:
train_df_1 = train_df
# # Tính ma trận tương quan
# cor_df = train_df.drop(['Date'], axis=1)
# correlation_matrix = cor_df.corr()

# # Hiển thị ma trận tương quan
# print(correlation_matrix)

In [62]:
def lag_transform_df(df: pd.DataFrame, lag: int):
    df_lag = df.copy()
    for col in df_lag.columns:
        for i in range(1, lag + 1):
            df_lag[f'{col}_lag_{i}'] = df_lag[col].shift(i)
        lag_cols = [f'{col}_lag_{i}' for i in range(1, lag + 1)]
        df_lag[f'{col}_mean'] = df_lag[lag_cols].mean(axis=1)
        df_lag[f'{col}_max'] = df_lag[lag_cols].max(axis=1)
        df_lag[f'{col}_min'] = df_lag[lag_cols].min(axis=1)
    return df_lag.dropna()

df_lag = lag_transform_df(train_df_1, 5)
print(df_lag.shape)
df_lag.head()

(1240, 45)


,open,high,low,close,volume,open_lag_1,open_lag_2,open_lag_3,open_lag_4,open_lag_5,...,close_max,close_min,volume_lag_1,volume_lag_2,volume_lag_3,volume_lag_4,volume_lag_5,volume_mean,volume_max,volume_min
5,7.60,7.73,7.57,7.71,2196358,8.34,7.68,7.44,6.99,7.86,...,7.76,7.47,846226.0,1305819.0,2771983.0,1956382.0,1103106.0,1596703.2,2771983.0,846226.0
6,7.71,7.73,7.68,7.68,1378687,7.60,8.34,7.68,7.44,6.99,...,7.71,7.47,2196358.0,846226.0,1305819.0,2771983.0,1956382.0,1815353.6,2771983.0,846226.0
7,7.71,7.73,7.65,7.68,1250077,7.71,7.60,8.34,7.68,7.44,...,7.71,7.52,1378687.0,2196358.0,846226.0,1305819.0,2771983.0,1699814.6,2771983.0,846226.0
8,7.65,7.68,7.63,7.63,937155,7.71,7.71,7.60,8.34,7.68,...,7.71,7.57,1250077.0,1378687.0,2196358.0,846226.0,1305819.0,1395433.4,2196358.0,846226.0
9,7.49,7.73,7.04,7.73,901146,7.65,7.71,7.71,7.60,8.34,...,7.71,7.57,937155.0,1250077.0,1378687.0,2196358.0,846226.0,1321700.6,2196358.0,846226.0


Shuffle and train test split

In [63]:
print(train_df.columns)

Index(['open', 'high', 'low', 'close', 'volume'], dtype='object')


In [64]:
Y = df_lag[train_df.columns].values
X = df_lag.drop(train_df.columns, axis=1).values
print(X.shape)
print(Y.shape)

(1240, 40)
(1240, 5)


Shuffle and train test split

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)






(992, 40)
(248, 40)
(992, 5)
(248, 5)


In [66]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score

model = LinearRegression()
model.fit(X_train, Y_train)

mse = mean_squared_error(Y_test, model.predict(X_test))
print(f"Mean Squared Error: {mse}")

mape = mean_absolute_percentage_error(Y_test, model.predict(X_test))
print(f"Mean Absolute Percentage Error: {mape}")

r2 = r2_score(Y_test, model.predict(X_test))
print(f"R2 Score: {r2}")





Mean Squared Error: 2692915347858.793
Mean Absolute Percentage Error: 0.09441464272679166
R2 Score: 0.8396260270381312
